<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/RandomForestEmotionClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Acoustic Features.csv")

In [50]:
df


,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy,is_train
0,relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,...,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970,True
1,relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,...,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967,False
2,relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,...,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963,True
3,relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,...,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968,True
4,relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,...,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,angry,0.174,0.578,5.265,161.144,2.985,0.065,0.718,0.046,0.265,...,0.838,0.413,0.125,0.323,0.140,0.120,2.691,0.867,0.969,True
396,angry,0.176,0.498,5.042,179.787,2.790,-0.148,0.342,-0.011,0.029,...,1.000,0.640,0.010,0.233,0.127,0.145,3.364,0.879,0.967,True
397,angry,0.187,0.557,4.724,134.032,1.672,0.566,0.880,-0.113,0.244,...,0.184,0.287,0.101,0.269,0.157,0.111,3.364,0.819,0.962,True
398,angry,0.140,0.573,4.470,113.600,1.997,-0.210,0.694,0.089,0.206,...,1.000,0.304,0.140,0.277,0.144,0.061,3.364,0.811,0.969,True


In [5]:
df.Class.value_counts()

relax    100
happy    100
sad      100
angry    100
Name: Class, dtype: int64

In [6]:
features = df.columns[1:]
features

Index(['_RMSenergy_Mean', '_Lowenergy_Mean', '_Fluctuation_Mean',
       '_Tempo_Mean', '_MFCC_Mean_1', '_MFCC_Mean_2', '_MFCC_Mean_3',
       '_MFCC_Mean_4', '_MFCC_Mean_5', '_MFCC_Mean_6', '_MFCC_Mean_7',
       '_MFCC_Mean_8', '_MFCC_Mean_9', '_MFCC_Mean_10', '_MFCC_Mean_11',
       '_MFCC_Mean_12', '_MFCC_Mean_13', '_Roughness_Mean', '_Roughness_Slope',
       '_Zero-crossingrate_Mean', '_AttackTime_Mean', '_AttackTime_Slope',
       '_Rolloff_Mean', '_Eventdensity_Mean', '_Pulseclarity_Mean',
       '_Brightness_Mean', '_Spectralcentroid_Mean', '_Spectralspread_Mean',
       '_Spectralskewness_Mean', '_Spectralkurtosis_Mean',
       '_Spectralflatness_Mean', '_EntropyofSpectrum_Mean',
       '_Chromagram_Mean_1', '_Chromagram_Mean_2', '_Chromagram_Mean_3',
       '_Chromagram_Mean_4', '_Chromagram_Mean_5', '_Chromagram_Mean_6',
       '_Chromagram_Mean_7', '_Chromagram_Mean_8', '_Chromagram_Mean_9',
       '_Chromagram_Mean_10', '_Chromagram_Mean_11', '_Chromagram_Mean_12',
      

In [7]:
#Creating Test and Train data
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
#View top 10 rows
df.head(10)

,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy,is_train
0,relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,...,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970,True
1,relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,...,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967,False
2,relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,...,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963,True
3,relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,...,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968,True
4,relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,...,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957,False
5,relax,0.137,0.510,7.664,116.044,2.313,-0.082,0.365,0.033,-0.136,...,0.391,0.102,0.000,0.280,0.240,0.355,1.682,0.662,0.958,True
6,relax,0.123,0.533,7.483,136.444,4.572,1.171,0.939,0.358,0.024,...,1.000,0.000,0.143,0.210,0.174,0.193,2.691,0.691,0.966,True
7,relax,0.103,0.575,7.006,141.132,3.495,0.553,0.275,0.269,0.257,...,1.000,0.030,0.746,0.366,0.267,-0.061,0.585,0.711,0.965,True
8,relax,0.145,0.542,6.846,82.048,4.010,0.550,0.568,0.559,0.218,...,0.150,0.910,0.000,0.322,0.210,0.031,0.408,0.796,0.970,True
9,relax,0.129,0.618,8.668,93.020,2.318,0.744,0.537,-0.116,0.119,...,0.018,0.385,0.000,0.388,0.340,0.122,1.682,0.627,0.965,False


In [8]:
#Create dataframes with train and test rows
train, test = df[df['is_train']==True], df[df['is_train']==False]

#show the number of rows in test and train dataframe
print("training data ", len(train))
print("testing data ", len(test))

training data  292
testing data  108


In [9]:
#Converting each Class naem into digits
y = pd.factorize(train['Class'])[0]
#viewing target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3])

In [10]:
#Converting each Class name into digits for test data
x = pd.factorize(test['Class'])[0]
#viewing target
x

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [11]:
target_names = (train['Class']).unique()
target_names


array(['relax', 'happy', 'sad', 'angry'], dtype=object)

In [12]:
#Creating Random Forest Classifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[features], y)


RandomForestClassifier(n_jobs=2, random_state=0)

In [13]:
#Applying trained Classifier to the test
predictions = clf.predict(test[features])
predictions

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 0, 2, 2, 0,
       2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 3, 2, 0, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3])

In [14]:
#viewing predicted probablities of first 10 observation
clf.predict_proba(test[features])[:10]

array([[0.77, 0.02, 0.13, 0.08],
       [0.66, 0.  , 0.29, 0.05],
       [0.31, 0.03, 0.64, 0.02],
       [0.49, 0.03, 0.44, 0.04],
       [0.6 , 0.07, 0.24, 0.09],
       [0.43, 0.11, 0.36, 0.1 ],
       [0.62, 0.02, 0.25, 0.11],
       [0.5 , 0.04, 0.34, 0.12],
       [0.79, 0.  , 0.16, 0.05],
       [0.52, 0.04, 0.19, 0.25]])

In [15]:
#mapping names of moods for each music for each predicted mood
preds  = target_names[clf.predict(test[features])]

#View the predicted moods for firat five observations
preds[10:20]

array(['relax', 'relax', 'relax', 'relax', 'relax', 'relax', 'relax',
       'relax', 'relax', 'relax'], dtype=object)

In [16]:
#Viewing Actual mood of test music or data
test['Class'].head()

1     relax
4     relax
9     relax
14    relax
16    relax
Name: Class, dtype: object

In [17]:
#Creating confusion matrix
#crosstab takes two sets of data and forms chart out of it
pd.crosstab(test['Class'], preds, rownames=['Actual mood'], colnames=['Predicted Species'])

Predicted Species,angry,happy,relax,sad
Actual mood,,,,
angry,22,0,1,2
happy,3,30,0,3
relax,0,0,21,4
sad,1,1,8,12


In [18]:
accuracy = accuracy_score(x, predictions) 

In [19]:
accuracy * 100

78.70370370370371

In [20]:
!git clone https://github.com/tiwaripari/ML.git

Cloning into 'ML'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 54 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), 1.59 MiB | 6.37 MiB/s, done.


In [21]:
from google.colab import files

# Prompt the user to upload a file
uploaded = files.upload()


Saving finalmusicfeatureextractednew.py to finalmusicfeatureextractednew.py


In [22]:
from finalmusicfeatureextractednew import feature_extraction

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
x = '/content/drive/MyDrive/TunePocket-Rock-Party-30-Sec-Intro-Preview (1).mp3'
features = {}
features = feature_extraction(x)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [33]:
features.columns

Index(['_RMSenergy_Mean', '_Lowenergy_Mean', '_Fluctuation_Mean',
       '_Tempo_Mean', '_MFCC_Mean_1', '_MFCC_Mean_2', '_MFCC_Mean_3',
       '_MFCC_Mean_4', '_MFCC_Mean_5', '_MFCC_Mean_6', '_MFCC_Mean_7',
       '_MFCC_Mean_8', '_MFCC_Mean_9', '_MFCC_Mean_10', '_MFCC_Mean_11',
       '_MFCC_Mean_12', '_MFCC_Mean_13', '_Roughness_Mean', '_Roughness_Slope',
       '_Zero-crossingrate_Mean', '_AttackTime_Mean', '_AttackTime_Slope',
       '_Rolloff_Mean', '_Eventdensity_Mean', '_Pulseclarity_Mean',
       '_Brightness_Mean', '_Spectralcentroid_Mean', '_Spectralspread_Mean',
       '_Spectralskewness_Mean', '_Spectralkurtosis_Mean',
       '_Spectralflatness_Mean', '_EntropyofSpectrum_Mean',
       '_Chromagram_Mean_1', '_Chromagram_Mean_2', '_Chromagram_Mean_3',
       '_Chromagram_Mean_4', '_Chromagram_Mean_5', '_Chromagram_Mean_6',
       '_Chromagram_Mean_7', '_Chromagram_Mean_8', '_Chromagram_Mean_9',
       '_Chromagram_Mean_10', '_Chromagram_Mean_11', '_Chromagram_Mean_12',
      

In [34]:
attribute_avg = train['_HarmonicChangeDetectionFunction_PeriodEntropy'].mean()


In [35]:
features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [36]:
prediction2 = target_names[clf.predict(features)]
prediction2

array(['angry'], dtype=object)

In [37]:
y = '/content/drive/MyDrive/Happiness.mp3'
features_2 = {}
features_2 = feature_extraction(y)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [38]:

features_2

,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,_MFCC_Mean_6,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,0.067174,50.148142,0.000006,184.570312,-278.565643,84.132698,-27.460295,-14.657911,-10.417986,0.084659,...,0.175449,0.331654,0.107137,0.133154,-0.000531,24.949371,-0.000531,0.028571,2636.463867,NaN


In [39]:
features_2['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [43]:
prediction3 = target_names[clf.predict(features_2)]
prediction3

array(['angry'], dtype=object)

In [42]:
z = '/content/Energetic.mp3'
features_3 = {}
features_3 = feature_extraction(z)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [46]:
features_3['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [47]:
prediction4 = target_names[clf.predict(features_3)]
prediction4

array(['relax'], dtype=object)